<a href="https://colab.research.google.com/github/sefeoglu/OntologyMatching-Evaluation_Results/blob/master/Graph_Model_Correct_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ontospy

     |████████████████████████████████| 7.6MB 3.9MB/s 
     |████████████████████████████████| 235kB 52.8MB/s 
     |████████████████████████████████| 870kB 61.8MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Created wheel for keepalive: filename=keepalive-0.5-cp37-none-any.whl size=8938 sha256=497ec848695f7daa57fabcebafb4074dce442b424137a2c46cb9fd7b846a7b14
  Stored in directory: /root/.cache/pip/wheels/dc/56/d1/1f5ce7b1976ed0030f39e69f33af490745441216b762be9a69
  Created wheel for rdflib-jsonld: filename=rdflib_jsonld-0.5.0-py2.py3-none-any.whl size=15372 sha256=464d1cf7af316fdf766a0be5e7facdb888b724b33e8233ab631adbab74d917f5
  Stored in directory: /root/.cache/pip/wheels/8d/e4/7f/9ebcb3e400c694e645c3adba40ef3e9bda78384ac3b9b0d13d
Successfully built keepalive rdflib-jsonld


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install sentence_transformers

     |████████████████████████████████| 92kB 3.1MB/s 
     |████████████████████████████████| 2.5MB 4.9MB/s 
     |████████████████████████████████| 1.2MB 16.4MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 901kB 18.7MB/s 
     |████████████████████████████████| 3.3MB 23.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=c104db081cc885519f51e1a0f8dcbe401fa530908d07d65f1e78f83981c5bc9a
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.14 which is incompatible.


## Preprocessing

### Ontology Parser

In [ ]:

import ontospy

class OntologyParser(object):
    
    def __init__(self, rdf_file):
        self.onto = ontospy.Ontospy(rdf_file)
        self.classes = self.get_classes()
        self.properties = self.get_properties()
    
    def get_properties(self):
        properties = []
        for prop in self.onto.all_properties:
            prop = str(prop).split("/")[-1].replace("*>","").split("#")[-1]
            properties.append(prop)
        return properties
   
    def get_classes(self):
        classes = []
        for concept in self.onto.all_classes:
            concept = str(concept).split("/")[-1].replace("*>","").split("#")[-1]
            classes.append(concept)
        return classes

    def get_all_triples(self):

        triples = self.onto.sparql("SELECT ?subject ?predicate ?object  WHERE { ?subject ?predicate ?object}")
        return triples

    def get_inferred_properties(self, class_uri):
        props = self.onto.getInferredPropertiesForClass(class_uri)
        return props

### GraphParser

In [ ]:
## preprocessing Codes

import sys, configparser
import os, itertools, re, logging, requests, urllib
import pandas as pd
import networkx as nx
from xml.dom import minidom
import itertools
import numpy as np
import logging
from scipy import spatial

from sentence_transformers import SentenceTransformer

# PACKAGE_PARENT = '.'
# SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser(__file__))))
# sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
# from OntologyParser import OntologyParser
try:
    # ubuntu works with USE(universal sentence encoder), otherwise it will run with bert sentence encoder
    import tensorflow_hub as hub
except:
    pass

flatten = lambda l: [item for sublist in l for item in sublist]
def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)
class GraphParser(object):

    def __init__(self, ontologies_in_alignment, alignments=None):
        ### Folder Definition ###

        self.prefix_path = "drive/MyDrive/Thesis_OM/"  #"/".join(os.path.dirname(os.path.abspath(__file__)).split("/")[:-2]) + "/"
        config = configparser.ConfigParser()
        config.read(self.prefix_path + 'config.ini')
        if alignments == None:
           self.alignments = [] 
        else:
            self.alignments = alignments
            
        self.ontologies_in_alignment = ontologies_in_alignment
        self.train_folder = self.prefix_path + str(config['Paths']['dataset_folder']) + str(config["General"]['dataset'])+'/ontologies/'
        self.clean_set = self.prefix_path + str(config["Paths"]["spreadsheet_folder"]) + str(config["Paths"]["clean_dataset"])
        self.preprocessed_dataset = self.prefix_path + str(config["Paths"]["spreadsheet_folder"]) + str(config["Paths"]["preprocessed_dataset"])
        self.alignment_folder = self.prefix_path + str(config['Paths']['dataset_folder']) + str(config["General"]['dataset']) + str(config["Paths"]["alignment_folder"])
        self.output_folder_csv = self.prefix_path + str(config["Paths"]["spreadsheet_folder"]) + str(config['Paths']["dataset_folder"])
        self.all_data_folder = self.prefix_path + str(config["Paths"]["spreadsheet_folder"])
        try:
            self.USE_link = "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed"
            self.USE = hub.load(self.USE_link)
        except:
            pass
        self.model_transformer = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
        self.stopwords = ["has"]

    def extractUSEEmbeddings(self, words):
        try:
            word_embeddings = self.USE(words).numpy()
        except:
            word_embeddings = self.model_transformer.encode(words)

        return word_embeddings

    def create_spreadsheet_from_triples(self):
        """Creates spreadsheet from triples
        """        
        file_list = os.listdir(self.train_folder)
        subject, predicate, object_ls, ns_l, ontology_list= [], [], [], [], []
        for file_name in file_list:
            ontology_list.append(self.train_folder+file_name)
            rdf_file = self.train_folder+file_name
            o_parser =  OntologyParser(rdf_file)
            triples = o_parser.get_all_triples()
            ns = file_name.split('.')[0]
            for s, p, o in triples:
                subject.append(s)
                predicate.append(p)
                object_ls.append(o)
                ns_l.append(ns)

        data = {'subject':subject, 'predicate':predicate, 'object': object_ls,'ns':ns_l}
        data_frame = pd.DataFrame(data)

        return data_frame
            
            
    def inferred_properties(self, ontology_parser):

        inferred_properties = []
        classes = ontology_parser.get_classes()
        for ent in classes:
            inferred_properties.append(ontology_parser.get_inferred_properties(ent.split('#')[1]))

        return inferred_properties

    def preprocess_triples(self, data_frame):   

        for i in range(len(data_frame)):
            if len(str(data_frame.predicate[i]).split('#')[-1].split("/")[-1].replace("*>","").split("#")[-1])!=0:
                clean_predicate = str(data_frame.predicate[i]).split('#')[-1].split("/")[-1].replace("*>","").split("#")[-1]
                data_frame.predicate[i] = clean_predicate
            if len(str(data_frame.subject[i]).split('#')[-1].split("/")[-1].replace("*>","").split("#")[-1])!=0:
                clean_subject = str(data_frame.subject[i]).split('#')[-1].split("/")[-1].replace("*>","").split("#")[-1]
                data_frame.subject[i] = clean_subject
            if len(str(data_frame.object[i]).split('#')[-1].split("/")[-1].replace("*>","").split("#")[-1])!=0:
                clean_object = str(data_frame.object[i]).split('#')[-1].split("/")[-1].replace("*>","").split("#")[-1]
                data_frame.object[i] = clean_object

        return data_frame

    def construct_neighbour_dicts(self,ontologies_in_alignment):

        neighbours_dicts_ent, neighbours_dicts_prop = {}, {}
        flatten = lambda l: [item for sublist in l for item in sublist]
        data_frame = self.data_frame 

        for ont in list(set(flatten(ontologies_in_alignment))):
            
            onto_frame = data_frame[data_frame.ns == ont.split('/')[-1].split('.')[0]]
            self.kg_df = pd.DataFrame({'source':onto_frame.subject, 'target':onto_frame.object, 'edge':onto_frame.predicate})
            
            neighbours_dicts_ent.update(self.entity_neighbour_dict(ont))

            neighbours_dicts_prop.update(self.property_neighbour_dict(ont))

        max_types = np.max([len([nbr_type for nbr_type in elem if flatten(nbr_type)]) for elem in neighbours_dicts_ent.values()])

        return neighbours_dicts_ent, neighbours_dicts_prop, max_types

    def entity_neighbour_dict(self, ontology):
        self.parser = OntologyParser(ontology)
        entities = self.parser.classes
        neighbours_dicts_ent = {}
        ont = str(ontology).split('/')[-1].split('.')[0]

        for ent in entities:
            subcClass_neighbour, equivalent_neighbour, domain_neighbour, range_neighbour, parents = self.one_hop_class(ent, ont)
            ent = ont+"#"+str(ent).split("/")[-1].replace("*>","").split("#")[-1]
            neighbours_dicts_ent[ent] = [[parents], subcClass_neighbour, domain_neighbour, range_neighbour, equivalent_neighbour]

        return neighbours_dicts_ent

    def property_neighbour_dict(self, ontology):
        
        parser = OntologyParser(ontology)
        properties = parser.properties
        neighbours_dicts_prop = {}
        ont = str(ontology).split('/')[-1].split('.')[0]

        for prop in properties:
            domain, _range, subprop, inverse_prop = self.one_hop_property(prop, ont)
            prop_node = ont+"#"+str(prop).split("/")[-1].replace("*>","").split("#")[-1]
            prop = ont+"#"+str(prop).split("/")[-1].replace("*>","").split("#")[-1]
            neighbours_dicts_prop[prop]=[[prop_node], domain, _range, subprop, inverse_prop]

        return neighbours_dicts_prop

    def one_hop_property(self, prop, ont, attention_type='two'):
        # get onProperty
        prop_range, prop_domain = [], []
        somevalues_df = self.kg_df[self.kg_df['edge']=='someValuesFrom'] 
        unionOf_df = self.kg_df[self.kg_df['edge']=='unionOf']
        oneof_df = self.kg_df[self.kg_df['edge']=='oneOf']
        allValuesFrom_df = self.kg_df[self.kg_df['edge']=='allValuesFrom']
        first_df = self.kg_df[self.kg_df['edge']=='first']
        rest_df = self.kg_df[self.kg_df['edge']=='rest']
        domain_df = self.kg_df[self.kg_df['edge']== 'domain']
        range_df = self.kg_df[self.kg_df['edge']== 'range']
       
        G = self.create_graph(domain_df)
        prop = prop.split('#')[-1]
        ### Domain of Property
        try:
            props =  nx.neighbors(G, prop)
            for p in props:
                if str(p).startswith('N'):
                    prop_domain = self.unionOf(p, unionOf_df, first_df, rest_df, prop_domain, ont)
                    prop_domain = self.Restriction(p, somevalues_df, first_df, rest_df, unionOf_df, prop_domain, ont)
                    prop_domain = self.Restriction(p, oneof_df, first_df, rest_df, unionOf_df, prop_domain, ont)
                    prop_domain = self.Restriction(p, allValuesFrom_df, first_df, rest_df, unionOf_df, prop_domain, ont)
                else:
                    prop_domain.append([ont+"#"+p])
        except:
            pass
        ### Range of Property###
        G = self.create_graph(range_df)
        try:
            props =  nx.neighbors(G, prop)
            for p in props:
                if str(p).startswith('N'):
                    prop_range = self.unionOf(p, unionOf_df, first_df, rest_df, prop_range, ont)
                    prop_range = self.Restriction(p, somevalues_df, first_df, rest_df, unionOf_df, prop_range, ont)
                    prop_range = self.Restriction(p, oneof_df, first_df, rest_df, unionOf_df, prop_range, ont)
                    prop_range = self.Restriction(p, allValuesFrom_df, first_df, rest_df, unionOf_df, prop_range, ont)
                else:
                    prop_range.append([ont+"#"+p])
        except:
            pass

        ## For subprop
        subprop, inverse_prop = [], []
        if attention_type == 'two':
            subprop, inverse_prop = self.two_hop_property(prop, subprop, inverse_prop, ont)
        
        if len(prop_domain) != 0:
            prop_domain =  prop_domain[0]
        if len(prop_range) != 0:
            prop_range =  prop_range[0]

        if len(subprop) != 0:
            subprop =  subprop[0]
        if len(inverse_prop) != 0:
            inverse_prop =  inverse_prop[0]

        return prop_domain, prop_range, subprop, inverse_prop

    def create_graph(self, data_frame):
        G=nx.from_pandas_edgelist(data_frame, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
        G = nx.to_undirected(G)
        return G

    def Restriction(self,node, main_df,first_df, rest_df, union_df, class_neighbour, ont):
        try:
            G = self.create_graph(main_df)
            allnodes = nx.neighbors(G, node)
            for anode in allnodes:
                if str(anode).startswith('N'):
                    class_neighbour = self.unionOf(anode,union_df,first_df, rest_df, class_neighbour, ont)
                    try:
                        G = self.create_graph(first_df)
                        first_nodes = nx.neighbors(G, anode)
                        for fn in first_nodes:
                            if str(fn).startswith('N'):
                                continue
                            else:
                                class_neighbour.append([ont+"#"+fn])
                        G = self.create_graph(rest_df)
                        rest_nodes = nx.neighbors(G, anode)
                        for rn in rest_nodes:
                            if str(rn).startswith('N'):
                                try:
                                    G = self.create_graph(first_df)
                                    fnodes2 = nx.neighbors(G, rn)
                                    for fn2 in fnodes2:
                                        class_neighbour.append([ont+"#"+fn2])
                                except:
                                    pass
                            else:
                                class_neighbour.append([ont+"#"+fn])
                    except:
                        pass

                else:
                    class_neighbour.append([ont+"#"+anode])
        except:
            pass
        return class_neighbour

    def unionOf(self, node, unionOf_df, first_df,rest_df, class_neighbour, ont):
        try:
            G = self.create_graph(unionOf_df)
            unodes = nx.neighbors(G, node)
            for unode in unodes:
                if str(unode).startswith('N'):
                    try:
                        G = self.create_graph(first_df)
                        first_nodes = nx.neighbors(G, unode)
                        for fn in first_nodes:
                            if str(fn).startswith('N'):
                                continue
                            else:
                                class_neighbour.append([ont+"#"+fn])
                        G = self.create_graph(rest_df)
                        rest_nodes = nx.neighbors(G, unode)
                        for rn in rest_nodes:
                            if str(rn).startswith('N'):
                                try:
                                    G = self.create_graph(first_df)
                                    fnodes2 = nx.neighbors(G, rn)
                                    for fn2 in fnodes2:
                                        class_neighbour.append([ont+"#"+fn2])
                                    # Rest of Rest
                                    G = self.create_graph(rest_df)
                                    rnodes2 = nx.neighbors(G, rn)
                                    for rn2 in rnodes2:
                                        if str(rn2).startswith('N'):
                                            try:
                                                G = self.create_graph(first_df)
                                                fnodes3 = nx.neighbors(G, rn2)
                                                for fn3 in fnodes3:
                                                    class_neighbour.append([ont+"#"+fn3])
                                            except:
                                                pass
                                        else:
                                            class_neighbour.append([ont+"#"+rn2])
                                except:
                                    pass
                            else:
                                class_neighbour.append([ont+"#"+fn])
                    except:
                        pass
                else:
                    class_neighbour.append([ont+"#"+unode])
        except:
            pass

        return class_neighbour

    def onProperty(self, rsnode, onProperty_df, class_neighbour, ont):

        try:
            G = self.create_graph(onProperty_df)
            nodes = nx.neighbors(G, rsnode)
            for node in nodes:
                class_neighbour.append([ont+"#"+node])
        except:
            pass

        return class_neighbour

    def range_of_prop(self, prop_node, range_df, class_neighbour, ont):
        try:
            G = self.create_graph(range_df)
            nodes = nx.neighbors(G, prop_node)
            for node in nodes:
                class_neighbour.append([ont+"#"+node])
        except:
            pass

        return class_neighbour
    
    def one_hop_class(self, entity, ont):
        ## get subClassOf and equivalentClass
        class_df = self.kg_df[self.kg_df['edge']=='subClassOf'] 
        equivalent_df = self.kg_df[self.kg_df['edge']=='equivalentClass']
        somevalues_df = self.kg_df[self.kg_df['edge']=='someValuesFrom'] 
        unionOf_df = self.kg_df[self.kg_df['edge']=='unionOf']
        oneof_df = self.kg_df[self.kg_df['edge']=='oneOf']
        allValuesFrom_df = self.kg_df[self.kg_df['edge']=='allValuesFrom']
        first_df = self.kg_df[self.kg_df['edge']=='first']
        rest_df = self.kg_df[self.kg_df['edge']=='rest']
        domain_df = self.kg_df[self.kg_df['edge']== 'domain']
        onProperty_df = self.kg_df[self.kg_df['edge']=='onProperty']
        range_df = self.kg_df[self.kg_df['edge']=='range']
        
        G = self.create_graph(class_df)
        subcClass_neighbour, domain_neighbour, equivalent_neighbour, range_neighbour = [], [], [], []
        ### subClassOf for children ###
        try:
            nodes = nx.neighbors(G, entity)
            for node in nodes:
                if str(node).startswith('N'):
                    subcClass_neighbour = self.unionOf(node, unionOf_df, first_df, rest_df, subcClass_neighbour, ont)
                    subcClass_neighbour = self.Restriction(node, somevalues_df, first_df, rest_df, unionOf_df, subcClass_neighbour, ont)
                    subcClass_neighbour = self.Restriction(node, oneof_df, first_df, rest_df, unionOf_df, subcClass_neighbour, ont)
                    subcClass_neighbour = self.Restriction(node, allValuesFrom_df, first_df, rest_df, unionOf_df, subcClass_neighbour, ont)
                    subcClass_neighbour = self.onProperty(node, onProperty_df, subcClass_neighbour, ont)
                else:
                    subcClass_neighbour.append([ont+"#"+node])
        except:
            pass

        ## path to root
        G=nx.from_pandas_edgelist(class_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
        all_parents = self.parser.onto.toplayer_classes
        parents = [] 
        for parent in all_parents:
            parent = str(parent).split("*")[1].replace("*>","").split("#")[-1].split("/")[-1]
            try:
                nodes = nx.astar_path(G, entity, parent)
                for node in nodes:
                    if str(node).startswith('N'):
                        parents = self.unionOf(node, unionOf_df, first_df, rest_df, parents, ont)
                        parents = self.Restriction(node, somevalues_df, first_df, rest_df, unionOf_df, parents, ont)
                        parents = self.Restriction(node, oneof_df, first_df, rest_df, unionOf_df, parents, ont)
                        parents = self.Restriction(node, allValuesFrom_df, first_df, rest_df, unionOf_df, parents, ont)
                        parents = self.onProperty(node, onProperty_df, parents, ont)
                    else:
                        parents.append([ont+"#"+node])
            except:
                pass
        
        if len(parents)!= 0:
            parent_formatted = []
            for parent in parents:
                parent_formatted.append(parent[0])
            parents = parent_formatted
        ## remove the parent class from children class
        for child_class in subcClass_neighbour:
            child_str = child_class[0]
            for parent in parents:
                if child_str == parent and len(subcClass_neighbour) != 0:
                    subcClass_neighbour.remove(child_class)
                    
        ####equivalentClass
        G = self.create_graph(equivalent_df)
        try:
            nodes = nx.neighbors(G, entity)
            for node in nodes:
                if str(node).startswith('N'):
                    equivalent_neighbour = self.unionOf(node, unionOf_df, first_df, rest_df, equivalent_neighbour, ont)
                    equivalent_neighbour = self.Restriction(node, somevalues_df, first_df, rest_df, unionOf_df, equivalent_neighbour, ont)
                    equivalent_neighbour = self.Restriction(node, oneof_df, first_df, rest_df, unionOf_df, equivalent_neighbour, ont)
                    equivalent_neighbour = self.Restriction(node, allValuesFrom_df, first_df, rest_df, unionOf_df, equivalent_neighbour, ont)
                    ## onProperty
                    equivalent_neighbour = self.onProperty(node, onProperty_df, equivalent_neighbour, ont)
                else:
                    equivalent_neighbour.append([ont+"#"+node])
        except:
            pass

        ## Domain and Range for props
        G = self.create_graph(range_df)
        try:
            nodes = nx.neighbors(G, entity)
            for node in nodes:
                range_neighbour.append([ont+"#"+node])
                try:
                    G = self.create_graph(domain_df)
                    domain_nodes = nx.neighbors(G, node)
                    for dn in domain_nodes:
                        if str(dn).startswith('N'):
                            range_neighbour = self.unionOf(dn, unionOf_df, first_df, rest_df, range_neighbour, ont) 
                            range_neighbour = self.Restriction(dn, somevalues_df, first_df, rest_df, unionOf_df, range_neighbour, ont)
                            range_neighbour = self.Restriction(dn, oneof_df, first_df, rest_df, unionOf_df, range_neighbour, ont)
                            range_neighbour = self.Restriction(dn, allValuesFrom_df, first_df, rest_df, unionOf_df, range_neighbour, ont)
                        else:
                            range_neighbour.append([ont+"#"+dn])
                except:
                    pass
                #find range of node
        except:
            pass
        ## domain probs
         
        G = self.create_graph(domain_df)
        try:
            nodes = nx.neighbors(G, entity)
            for node in nodes:
                domain_neighbour.append([ont+"#"+node])
                try:
                    G = self.create_graph(range_df)
                    range_nodes = nx.neighbors(G, node)
                    for rn in range_nodes:
                        if str(rn).startswith("N"):
                            domain_neighbour = self.unionOf(rn, unionOf_df, first_df, rest_df, domain_neighbour, ont)
                            domain_neighbour = self.Restriction(rn, somevalues_df, first_df, rest_df, unionOf_df, domain_neighbour, ont)
                            domain_neighbour = self.Restriction(rn, oneof_df, first_df, rest_df, unionOf_df, domain_neighbour, ont)
                            domain_neighbour = self.Restriction(rn, allValuesFrom_df, first_df, rest_df, unionOf_df, domain_neighbour, ont)
                        else:
                            domain_neighbour.append([ont+"#"+rn])
                except:
                    pass
                #find range of node
        except:
            pass
        
        return subcClass_neighbour, equivalent_neighbour, domain_neighbour, range_neighbour, parents

    def two_hop_property(self, prop_node, subprop, inverse_prop, ont):

        subProperty_df = self.kg_df[self.kg_df['edge']== 'subPropertyOf']
        inverse_df = self.kg_df[self.kg_df['edge']== 'inverseOf']
        ##subPropertyOf
        try:
            G = self.create_graph(subProperty_df)
            nodes = nx.neighbors(G, prop_node)
            for node in nodes:
                subprop.append([ont+"#"+node])
        except:
            pass
        ### inverseOf
        try:
            G = self.create_graph(inverse_df)
            nodes = nx.neighbors(G, prop_node)
            for node in nodes:
                inverse_prop.append([ont+"#"+node])
        except:
            pass
        return subprop, inverse_prop

    def cleaning_ontology_elements(self, data_frame):
        """ Clean data from ontology elements without semantic relations regarding concepts
        """        
        full_data = self.object_cleanup(data_frame)
        full_data = self.predicate_cleanup(full_data)
        full_data.drop_duplicates(keep='first',inplace=True) 

        return full_data

    def object_cleanup(self, full_data):
        
        full_data = full_data[full_data.object.str.startswith('nil') == False]
        full_data = full_data[full_data.object.str.startswith('All') == False]
        full_data = full_data[full_data.object.str.startswith('Ontology') == False]
        full_data = full_data[full_data.object.str.startswith('Thing') == False]
        # Restriction
        full_data = full_data[full_data.object.str.startswith('Rest') == False]
        
        return full_data

    def predicate_cleanup(self,full_data ):
        predicate_eliminator = ['cardinality', 'comment','complementOf','maxCardinality', 'minCardinality',
                       'qualifiedCardinality','versionInfo', 'hasValue', 'disjointWith']
        clean_full_data = full_data
        for eliminator in predicate_eliminator:
            clean_full_data = clean_full_data[clean_full_data.predicate != eliminator]
        # There is a problem regarding hasValue.
        clean_full_data = clean_full_data[clean_full_data.predicate != 'hasValue']
        return clean_full_data

    def cleaning_process(self):
        data_frame = self.create_spreadsheet_from_triples()
        data_frame = self.preprocess_triples(data_frame)
        self.data_frame = self.cleaning_ontology_elements(data_frame)

        return self.data_frame

    def generate_mappings(self, ontologies_in_alignment, gt_mappings):
        
        ent_mappings, prop_mappings = [], []

        for l in ontologies_in_alignment:

            ont1 = l[0]
            ont2 = l[1]
            parser1 = OntologyParser(ont1)
            parser2 = OntologyParser(ont2)
            ent1 = parser1.classes
            ent2 = parser2.classes
            prop1 = parser1.properties
            prop2 = parser2.properties
            ent_mapping = list(itertools.product(ent1, ent2))

            prop_mapping = list(itertools.product(prop1, prop2))

            pre1 = l[0].split("/")[-1].rsplit(".",1)[0].replace("-", "_")
            pre2 = l[1].split("/")[-1].rsplit(".",1)[0].replace("-", "_")

            ent_mappings.extend([(str(pre1) + "#" + str(el[0]), str(pre2) + "#" + str(el[1])) for el in ent_mapping])
            prop_mappings.extend([(str(pre1) + "#" + str(el[0]), str(pre2) + "#" + str(el[1])) for el in prop_mapping])

        if gt_mappings:
            data_ent = {mapping: False for mapping in ent_mappings}
            data_prop = {mapping: False for mapping in prop_mappings}
            s_ent = set(ent_mappings)
            s_prop = set(prop_mappings)

            for mapping in set(gt_mappings):
                if mapping in s_ent:
                    data_ent[mapping] = True
                elif mapping in s_prop:
                    data_prop[mapping] = True
                else:
                    mapping = tuple([el.replace(",-", "_") for el in mapping])
                    if mapping in s_ent:
                        data_ent[mapping] = True
                    elif mapping in s_prop:
                        data_prop[mapping] = True
                    else:
                        logging.info ("Warning: {} given in alignments could not be found in source/target ontology.".format(mapping))
                        continue
            return (data_ent, data_prop)

        return (ent_mappings, prop_mappings)

    def run_abbreviation_resolution(self, inp, filtered_dict):
        # Resolving abbreviations to full forms
        logging.info ("Resolving abbreviations...")
        inp_resolved = []
        for concept in inp:
            for key in filtered_dict:
                concept = concept.replace(key, filtered_dict[key])
            final_list = []
            # Lowering case except in abbreviations
            for word in concept.split(" "):
                if not re.search("[A-Z][A-Z]+", word):
                    final_list.append(word.lower())
                else:
                    final_list.append(word)
            concept = " ".join(final_list)
            inp_resolved.append(concept)

        return inp_resolved

    def run_spellcheck(self, inp_resolved):
        # Spelling checker and corrector
        logging.info ("Running spellcheck...")

        url = "https://grammarbot.p.rapidapi.com/check"

        headers = {
            'x-rapidapi-host': "grammarbot.p.rapidapi.com",
            'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62",
            'content-type': "application/x-www-form-urlencoded"
            }

        inp_spellchecked = []

        for concept in inp_resolved:
            payload = "language=en-US&text=" + urllib.parse.quote_plus(concept)
            response = requests.request("POST", url, data=payload, headers=headers).json()
            concept_corrected = str(concept)
            
            for elem in response["matches"]:
                start, end = elem["offset"], elem["offset"] + elem["length"]
                concept_corrected = concept_corrected[:start] + elem["replacements"][0]["value"] + concept_corrected[end:]
            
            if concept.lower() != concept_corrected.lower():
                logging.info ("{} corrected to {}".format(concept, concept_corrected))
                inp_spellchecked.append(concept_corrected)
            else:
                inp_spellchecked.append(concept)

        return inp_spellchecked


    def construct_abbreviation_resolution_dict(self, all_mappings):
        # Constructs an abbrevation resolution dict
        logging.info ("Constructing abbrevation resolution dict....")
    
        final_dict = {}
        for mapping in all_mappings:
            mapping = tuple([ str(el).split("#")[1] for el in mapping])
            is_abb = re.search("[A-Z][A-Z]+", mapping[0])
            if is_abb:
                abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
                if is_abb.group() in abbreviation:
                    
                    start = abbreviation.find(is_abb.group())
                    end = start + len(is_abb.group())
                    fullform = "_".join(mapping[1].split("_")[start:end])
                    
                    rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
                    rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
                    if is_abb.group() not in final_dict:
                        final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
                    else:
                        final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

            is_abb = re.search("[A-Z][A-Z]+", mapping[1])
            if is_abb:
                abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
                
                if is_abb.group() in abbreviation:
                    start = abbreviation.find(is_abb.group())
                    end = start + len(is_abb.group())
                    fullform = "_".join(mapping[0].split("_")[start:end])

                    rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
                    rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
                    if is_abb.group() not in final_dict:
                        final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
                    else:
                        final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

        keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
        abb_embeds = dict(zip(keys, self.extractUSEEmbeddings(keys)))

        scored_dict = {}
        for abbr in final_dict:
            sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                        else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
            scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

        resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
        filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
        logging.info ("Results after abbreviation resolution: ", filtered_dict)
        return filtered_dict

    def camel_case_split(self, identifier):
        # Splits camel case strings
        matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
        return [m.group(0) for m in matches]

    def parse(self, word):
        return " ".join(flatten([el.split("_") for el in self.camel_case_split(word)]))

    def extract_keys(self, ontologies_in_alignment):

        extracted_elems = []
        data_frame = self.data_frame 
        for ont_name in list(set(flatten(ontologies_in_alignment))):
            ont_name_filt = ont_name.split("/")[-1].rsplit(".",1)[0].replace("-", "_")
            onto_frame = data_frame[data_frame.ns == ont_name_filt]
            self.predicate = list(onto_frame["predicate"])
            self.subject = list(onto_frame["subject"])
            self.object =list(onto_frame["object"])
            extracted_elems.extend([ont_name_filt + "#" + elem for elem in self.subject + self.predicate + self.object])
        extracted_elems = list(set(extracted_elems))
        inp = []
        for word in extracted_elems:
            ont_name = word.split("#")[0]
            elem = word.split("#")[1]
            inp.append(self.parse(elem))

        logging.info ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

        extracted_elems = ["<UNK>"] + extracted_elems
        return inp, extracted_elems


    def extract_embeddings(self, inp, extracted_elems):

        # Creates embeddings to index dict, word to index dict etc
        embeds = np.array(self.extractUSEEmbeddings(inp))
        embeds = np.array([np.zeros(embeds.shape[1],)] + list(embeds))
        embeddings = dict(zip(extracted_elems, embeds))

        emb_vals = list(embeddings.values())
        emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
        emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}

        return emb_vals, emb_indexer, emb_indexer_inv

    def remove_stopwords(self, inp):
        # Remove high frequency stopwords
        inp_filtered = []
        for elem in inp:
            words = " ".join([word for word in elem.split() if word not in self.stopwords])
            words = words.replace("-", " ")
            inp_filtered.append(words)
        return inp_filtered

    def process(self, spellcheck=False):
        self.data_frame = self.cleaning_process()
        gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in self.alignments]
        gt_mappings = [tuple([el.split("#")[0].rsplit(".", 1)[0] +  "#" +  el.split("#")[1] for el in tup]) for tup in gt_mappings]

        ent_mappings, prop_mappings = self.generate_mappings(self.ontologies_in_alignment, gt_mappings) #OK
        
        inp, extracted_elems = self.extract_keys(self.ontologies_in_alignment)
        filtered_dict = self.construct_abbreviation_resolution_dict(list(ent_mappings) + list(prop_mappings))

        inp_resolved = self.run_abbreviation_resolution(inp, filtered_dict)
        if spellcheck:
            try:
                inp_resolved = self.run_spellcheck(inp_resolved)
            except:
                pass
        inp = self.remove_stopwords(inp_resolved)
        emb_vals, emb_indexer, emb_indexer_inv = self.extract_embeddings(inp, extracted_elems)
        neighbours_dicts_ent, neighbours_dicts_prop, max_types = self.construct_neighbour_dicts(self.ontologies_in_alignment)
   
        return  ent_mappings, prop_mappings, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts_ent, neighbours_dicts_prop, max_types


## Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SemanticAttention(nn.Module):

    def __init__(self, in_size, hidden_size=768):

        super(SemanticAttention, self).__init__()

        self.project = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )

    def forward(self, z):

        w = self.project(z).mean(0)                 # (M, 1)
        beta = torch.softmax(w, dim=0)                 # (M, 1)
        beta = beta.expand((z.shape[0],) + beta.shape) # (N, M, 1)

        result =  (beta * z)                     # (N, D * K)
        return result


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


## Pytorch Graph Attention Network
### the original codes here: https://github.com/Diego999/pyGAT and we exploited from their codes.
#This is a pytorch implementation of the Graph Attention Network (GAT) model presented by Veličković et. al (2017, https://arxiv.org/abs/1710.10903).


class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
        a_input = self._prepare_attentional_mechanism_input(Wh)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
        zero_vec = -9e15*torch.ones_like(e)
        adj = torch.DoubleTensor(adj).to(self.device)
        attention = torch.where(adj > 0, e, zero_vec)


        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)

        h_prime = torch.matmul(attention, Wh)
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        N = Wh.size()[0] # number of nodes

        # Below, two matrices are created that contain embeddings in their rows in different orders.
        # (e stands for embedding)
        # These are the rows of the first matrix (Wh_repeated_in_chunks): 
        # e1, e1, ..., e1,            e2, e2, ..., e2,            ..., eN, eN, ..., eN
        # '-------------' -> N times  '-------------' -> N times       '-------------' -> N times
        # 
        # These are the rows of the second matrix (Wh_repeated_alternating): 
        # e1, e2, ..., eN, e1, e2, ..., eN, ..., e1, e2, ..., eN 
        # '----------------------------------------------------' -> N times
        # 
        
        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=0)
        Wh_repeated_alternating = Wh.repeat(N, 1)
        # Wh_repeated_in_chunks.shape == Wh_repeated_alternating.shape == (N * N, out_features)

        # The all_combination_matrix, created below, will look like this (|| denotes concatenation):
        # e1 || e1
        # e1 || e2
        # e1 || e3
        # ...
        # e1 || eN
        # e2 || e1
        # e2 || e2
        # e2 || e3
        # ...
        # e2 || eN
        # ...
        # eN || e1
        # eN || e2
        # eN || e3
        # ...
        # eN || eN

        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
        # all_combinations_matrix.shape == (N * N, 2 * out_features)

        return all_combinations_matrix.view(N, N, 2 * self.out_features)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'




In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os, sys
import numpy as np

# PACKAGE_PARENT = '.'
# SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser(__file__))))
# sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))


# from graph_attention import GraphAttentionLayer as GATConv


class HANLayer(nn.Module):
 
    def __init__(self, in_size, out_size, dropout):
        super().__init__()
        self.alpha = 0.2
        self.gat = GraphAttentionLayer(in_size, out_size, dropout, alpha=self.alpha, concat=True)
        self.semantic_attention = SemanticAttention(in_size, out_size)
        
    def forward(self, hs, adj):
        
        semantic_embeddings = []
        for i, h in enumerate(hs):
            semantic_embeddings.append(self.gat(h, adj[i]))
        semantic_embeddings = torch.stack(semantic_embeddings)

        
        return semantic_embeddings                

class SiamHAN(nn.Module):

    def __init__(self, emb_vals, max_types, max_paths, max_pathlen, threshold=0.9):
      
        super().__init__()
        self.dropout = 0.6
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.n_neighbours = max_types
        self.max_types = nn.Parameter(torch.DoubleTensor([max_types]))
        self.max_types.requires_grad = False
        self.max_paths = max_paths
        self.max_pathlen = max_pathlen
        self.embedding_dim = np.array(emb_vals).shape[1]
       
        self.threshold = nn.Parameter(torch.DoubleTensor([threshold]))
        self.threshold.requires_grad = False
        self.gnn = HANLayer(self.embedding_dim, self.embedding_dim, self.dropout)
        
        ### Embedding layer
        self.name_embedding = nn.Embedding(len(emb_vals), self.embedding_dim)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
        self.name_embedding.weight.requires_grad = False
        ## output layer
        self.output = nn.Linear(2*self.embedding_dim, 300)
        ## similarity score
        self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
        
        self.weight = nn.Parameter(torch.DoubleTensor([1.0]))

        self.weight_prop = nn.Parameter(torch.DoubleTensor([0.33]))
        self.v = nn.Parameter(torch.DoubleTensor([1/(self.max_pathlen+1) for i in range(self.max_pathlen+1)]))
  
  
    def forward(self, nodes, features, prop_nodes, prop_features, max_prop_len):
        # define here again for ontologies
        # get node and feature embedding
        results = []
        
        self.max_prop_len = max_prop_len
        nodes = nodes.permute(1,0) # dim: (2, batch_size)
        features = features.permute(1,0,2,3,4) # dim: (2, batch_size, max_types, max_paths, max_pathlen)
 
        for i in range(2):
            node_emb = self.name_embedding(nodes[i]) # dim: (2, batch_size)
            feature_emb = self.name_embedding(features[i]) #  dim: (2, batch_size, max_types, max_paths, max_pathlen, 512)
         
            h_primes = []

            for j, _node_emb in enumerate(node_emb):
                h, adj = self.create_adjacency_and_homgraph(feature_emb[j], _node_emb)
                semantic_attention_out = self.gnn(h, adj)
                semantic_attention_out = torch.sum((self.v[None,:,None] * semantic_attention_out), dim=1)
                if self.n_neighbours == 2:
                    h_prime = self.weight*semantic_attention_out[0,:]\
                        + self.weight*semantic_attention_out[1,:]
                    h_primes.append(h_prime)

                if self.n_neighbours == 3:
                    h_prime = self.weight*semantic_attention_out[0,:]\
                        + self.weight*semantic_attention_out[1,:]\
                        +self.weight*semantic_attention_out[2,:]
                    h_primes.append(h_prime)
                if self.n_neighbours == 4:
                    h_prime = self.weight*semantic_attention_out[0,:]\
                        + self.weight*semantic_attention_out[1,:]\
                        +self.weight*semantic_attention_out[2,:]\
                        +self.weight*semantic_attention_out[3,:]
                    h_primes.append(h_prime)
                if self.n_neighbours == 5:
                    h_prime = self.weight*semantic_attention_out[0,:]\
                        + self.weight*semantic_attention_out[1,:]\
                        +self.weight*semantic_attention_out[2,:]\
                        +self.weight*semantic_attention_out[3,:]\
                        +self.weight*semantic_attention_out[4,:]
                    h_primes.append(h_prime)
            h_primes = torch.stack(h_primes)
            context = torch.cat((node_emb, h_primes), dim=1)
            contextual = self.output(context)
            results.append(contextual)
        ## consine similarity layer which computes the similarity score
                
        sim_ent = self.cosine_sim_layer(results[0], results[1])

        if prop_nodes.nelement() != 0:
            # Calculate prop sum
            aggregated_prop_sum = torch.sum(self.name_embedding(prop_features), dim=-2)
            sim_prop = self.weight_prop * self.cosine_sim_layer(aggregated_prop_sum[:,0,0], aggregated_prop_sum[:,1,0])
            sim_prop += self.weight_prop * self.cosine_sim_layer(aggregated_prop_sum[:,0,1], aggregated_prop_sum[:,1,1])
            sim_prop += (1.0-2*self.weight_prop) * self.cosine_sim_layer(aggregated_prop_sum[:,0,2], aggregated_prop_sum[:,1,2])
 
            

            return torch.cat((sim_ent, sim_prop))
            
        return sim_ent

    def create_adjacency_and_homgraph(self, features_list,  node):

        adj, hs = [], []

        for _, features in enumerate(features_list):
            h, g =  self.create_graph_freatures(features, node)
            hs.append(h)
            adj.append(g)
        return hs, adj

    def create_adj(self, size):

        adj = np.zeros(shape=(size, size))
        adj[0,:] = 1.0

        return adj

    def create_graph_freatures(self, features, c_node):
        h = []
        values = [feature.detach().to("cpu").numpy()[0,0] for feature in features]
        if not all(np.array(values) == 0):
            h.append(c_node.detach().to("cpu").numpy())
            
            for i, feature in enumerate(features):
                for j, feature_node in enumerate(feature):
                    h.append(feature_node.detach().to("cpu").numpy())
                break
        else:
            for j in range(self.max_pathlen+1):
                h.append(np.zeros(self.embedding_dim))
        adj = self.create_adj(len(h))
        h_tensor = torch.DoubleTensor(h).to(self.device)
        return h_tensor, adj

## Trainer

In [14]:
## Trainer Codes
import configparser, logging, random, sys, pickle
import numpy as np
from collections import OrderedDict
from math import ceil
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F


# Load reference alignments 
ontologies_in_alignment = []
def load_alignments(folder):
    global ontologies_in_alignment
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2')))
        src = train_folder + doc.getElementsByTagName('Ontology')[0].getAttribute("rdf:about").split("/")[-1].rsplit(".", 1)[0] + ".owl"
        targ = train_folder + doc.getElementsByTagName('Ontology')[1].getAttribute("rdf:about").split("/")[-1].rsplit(".", 1)[0] + ".owl"
        ontologies_in_alignment.append((src, targ))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource')) for (a,b) in ls])
    return alignments

prefix_path = "drive/MyDrive/Thesis_OM/"#"/".join(os.path.dirname(os.path.abspath(__file__)).split("/")[:-1]) + "/"

print ("Prefix path: ", prefix_path)

# Read `config.ini` and initialize parameter values
config = configparser.ConfigParser()
config.read(prefix_path+'config.ini')

# Initialize variables from config

quick_mode = str(config["General"]["quick_mode"])

K = int(config["General"]["K"])
ontology_split = str(config["General"]["ontology_split"]) == "True"
max_false_examples = int(config["General"]["max_false_examples"])

alignment_folder = prefix_path + "datasets/" + str(config["General"]["dataset"]) + "/alignments/"
train_folder = prefix_path + "datasets/" + str(config["General"]["dataset"]) + "/ontologies/"
cached_embeddings_path = prefix_path + str(config["Paths"]["embedding_cache_path"])
model_path = prefix_path + str(config["Paths"]["save_model_path"])

spellcheck = config["Preprocessing"]["has_spellcheck"] == "True"

max_paths = int(config["Parameters"]["max_paths"])
max_pathlen = int(config["Parameters"]["max_pathlen"])
bag_of_neighbours = config["Parameters"]["bag_of_neighbours"] == "True"
weighted_sum = config["Parameters"]["weighted_sum"] == "True"

lr = float(config["Hyperparameters"]["lr"])
num_epochs = int(config["Hyperparameters"]["num_epochs"])
weight_decay = float(config["Hyperparameters"]["weight_decay"])
batch_size = int(config["Hyperparameters"]["batch_size"])

reference_alignments = load_alignments(alignment_folder)
gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]
gt_mappings = [tuple([el.split("#")[0].rsplit(".", 1)[0] +  "#" +  el.split("#")[1] for el in tup]) for tup in gt_mappings]
print ("Ontologies being aligned are: ", ontologies_in_alignment)

# Preprocessing and parsing input data for training
preprocessing = GraphParser(ontologies_in_alignment, reference_alignments)
data_ent, data_prop, emb_indexer_new, emb_indexer_inv_new, emb_vals_new, neighbours_dicts_ent, neighbours_dicts_prop, max_types = preprocessing.process(spellcheck=True)
        
if os.path.isfile(cached_embeddings_path):
    print("Found cached embeddings...")
    emb_indexer_cached, emb_indexer_inv_cached, emb_vals_cached = pickle.load(open(cached_embeddings_path, "rb"))
else:
    emb_indexer_cached, emb_indexer_inv_cached, emb_vals_cached = {}, {}, []

emb_vals, emb_indexer, emb_indexer_inv = list(emb_vals_cached), dict(emb_indexer_cached), dict(emb_indexer_inv_cached)

s = set(emb_indexer.keys())
idx = len(emb_indexer_inv)
for term in emb_indexer_new:
    if term not in s:
        emb_indexer[term] = idx
        emb_indexer_inv[idx] = term
        emb_vals.append(emb_vals_new[emb_indexer_new[term]])
        idx += 1

direct_inputs, direct_targets = [], []
threshold_results = {}

def optimize_threshold():
    '''
    Function to optimise threshold on validation set.
    Calculates performance metrics (precision, recall, F1-score, F2-score, F0.5-score) for a
    range of thresholds, dictated by the range of scores output by the model, with step size 
    0.001 and updates `threshold_results` which is the relevant dictionary.
    '''
    global val_data_t_ent, val_data_f_ent, threshold_results, batch_size, direct_inputs, direct_targets
    all_results = OrderedDict()
    direct_inputs, direct_targets = [], []
    with torch.no_grad():
        all_pred = []
        
        np.random.shuffle(val_data_t_ent)
        np.random.shuffle(val_data_f_ent)

        np.random.shuffle(val_data_t_prop)
        np.random.shuffle(val_data_f_prop)

        # Create two sets of inputs: one for entities and one for properties
        inputs_pos, nodes_pos, targets_pos = generate_input(val_data_t_ent, 1, neighbours_dicts_ent)
        inputs_neg, nodes_neg, targets_neg = generate_input(val_data_f_ent, 0, neighbours_dicts_ent)
        inputs_pos_prop, nodes_pos_prop, targets_pos_prop = generate_input(val_data_t_prop, 1, neighbours_dicts_prop)
        inputs_neg_prop, nodes_neg_prop, targets_neg_prop = generate_input(val_data_f_prop, 0, neighbours_dicts_prop)

        inputs_all = list(inputs_pos) + list(inputs_neg)
        targets_all = list(targets_pos) + list(targets_neg)
        nodes_all = list(nodes_pos) + list(nodes_neg)
        
        all_inp = list(zip(inputs_all, targets_all, nodes_all))
        all_inp_shuffled = random.sample(all_inp, len(all_inp))
        inputs_all, targets_all, nodes_all = list(zip(*all_inp_shuffled))

        inputs_all_prop = list(inputs_pos_prop) + list(inputs_neg_prop)
        targets_all_prop = list(targets_pos_prop) + list(targets_neg_prop)
        nodes_all_prop = list(nodes_pos_prop) + list(nodes_neg_prop)
        
        all_inp_prop = list(zip(inputs_all_prop, targets_all_prop, nodes_all_prop))
        all_inp_shuffled_prop = random.sample(all_inp_prop, len(all_inp_prop))
        if all_inp_shuffled_prop:
            inputs_all_prop, targets_all_prop, nodes_all_prop = list(zip(*all_inp_shuffled_prop))
        else:
            inputs_all_prop, targets_all_prop, nodes_all_prop = [], [], []

        if len(inputs_all_prop) == 0:
            max_prop_len = 0
        else:
            max_prop_len = np.max([[[len(elem) for elem in prop] for prop in elem_pair] 
            for elem_pair in inputs_all_prop])

        batch_size = min(batch_size, len(inputs_all))
        num_batches = int(ceil(len(inputs_all)/batch_size))
        batch_size_prop = int(ceil(len(inputs_all_prop)/num_batches))
        
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size
            batch_start_prop = batch_idx * batch_size_prop
            batch_end_prop = (batch_idx+1) * batch_size_prop

            inputs = np.array(to_feature(inputs_all[batch_start: batch_end]))
            targets = np.array(targets_all[batch_start: batch_end])
            nodes = np.array(nodes_all[batch_start: batch_end])

            inputs_prop = np.array(pad_prop(inputs_all_prop[batch_start_prop: batch_end_prop], max_prop_len))
            targets_prop = np.array(targets_all_prop[batch_start_prop: batch_end_prop])
            nodes_prop = np.array(nodes_all_prop[batch_start_prop: batch_end_prop])
            
            targets = np.concatenate((targets, targets_prop), axis=0)

            inp_elems = torch.LongTensor(inputs).to(device)
            node_elems = torch.LongTensor(nodes).to(device)
            targ_elems = torch.DoubleTensor(targets).to(device)

            inp_props = torch.LongTensor(inputs_prop).to(device)
            node_props = torch.LongTensor(nodes_prop).to(device)

            # Run model on entities and properties 
            outputs = model(node_elems, inp_elems, node_props, inp_props, max_prop_len)
            outputs = [el.item() for el in outputs]
            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                if idx < len(nodes):
                    ent1 = emb_indexer_inv[nodes[idx][0]]
                    ent2 = emb_indexer_inv[nodes[idx][1]]
                else:
                    ent1 = emb_indexer_inv[nodes_prop[idx-len(nodes)][0]]
                    ent2 = emb_indexer_inv[nodes_prop[idx-len(nodes)][1]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
        direct_targets = [True if el else False for el in direct_targets]
        
        print ("Len (direct inputs): ", len(direct_inputs))
        for idx, direct_input in enumerate(direct_inputs):
            ent1 = emb_indexer_inv[direct_input[0]]
            ent2 = emb_indexer_inv[direct_input[1]]
            sim = cos_sim(emb_vals[direct_input[0]], emb_vals[direct_input[1]])
            all_results[(ent1, ent2)] = (round(sim, 3), direct_targets[idx])
        
        # Low threshold is lowest value output by model and high threshold is the highest value
        low_threshold = round(np.min([el[0] for el in all_results.values()]) - 0.02, 3)
        high_threshold = round(np.max([el[0] for el in all_results.values()]) + 0.02, 3)
        threshold = low_threshold
        step = 0.001

        if not val_data_t_prop:
            val_data_t_tot = val_data_t_ent
        else:
            val_data_t_tot = [tuple(pair) for pair in np.concatenate((val_data_t_ent, val_data_t_prop), axis=0)]
        # Iterate over every threshold with step size of 0.001 and calculate all evaluation metrics
        while threshold < high_threshold:
            threshold = round(threshold, 3)
            res = []
            for i,key in enumerate(all_results):
                if all_results[key][0] > threshold:
                    res.append(key)
            s = set(res)
            fn_list = [(key, all_results[key][0]) for key in val_data_t_tot if key not in s]
            fp_list = [(elem, all_results[elem][0]) for elem in res if not all_results[elem][1]]
            tp_list = [(elem, all_results[elem][0]) for elem in res if all_results[elem][1]]
            
            tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
            exception = False
            
            try:
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1score = 2 * precision * recall / (precision + recall)
                f2score = 5 * precision * recall / (4 * precision + recall)
                f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
            except:
                exception = True
                step = 0.001
                threshold += step
                continue

            if threshold in threshold_results:
                threshold_results[threshold].append([precision, recall, f1score, f2score, f0_5score])
            else:
                threshold_results[threshold] = [[precision, recall, f1score, f2score, f0_5score]]
            threshold += step

 

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

def generate_data_neighbourless(elem_tuple):
    return [emb_indexer[elem] for elem in elem_tuple]

def embedify(seq):
    for item in seq:
        if isinstance(item, list):
            yield list(embedify(item))
        else:
            yield emb_indexer[item]

def pad_prop(inputs, max_prop_len):
    inputs_padded = [[[elem + [0 for i in range(max_prop_len - len(elem))]
                         for elem in prop]
                    for prop in elem_pair]
                for elem_pair in inputs]
    return inputs_padded

def generate_data(elem_tuple, neighbours_dicts):
    return list(embedify([neighbours_dicts[elem] for elem in elem_tuple]))

def to_feature(inputs):
    inputs_lenpadded = [[[[path[:max_pathlen] + [0 for i in range(max_pathlen -len(path[:max_pathlen]))]
                                    for path in nbr_type[:max_paths]]
                                for nbr_type in ent[:max_types]]
                            for ent in elem]
                        for elem in inputs]
    inputs_pathpadded = [[[nbr_type + [[0 for j in range(max_pathlen)]
                             for i in range(max_paths - len(nbr_type))]
                            for nbr_type in ent] for ent in elem]
                        for elem in inputs_lenpadded]
    return inputs_pathpadded

def generate_input(elems, target, neighbours_dicts):
    inputs, targets, nodes = [], [], []
    global direct_inputs, direct_targets
    for elem in list(elems):
        try:
            inputs.append(generate_data(elem, neighbours_dicts))
            nodes.append(generate_data_neighbourless(elem))
            targets.append(target)
        except KeyError as e:
            direct_inputs.append(generate_data_neighbourless(elem))
            direct_targets.append(target)
        except Exception as e:
            raise
    return inputs, nodes, targets

def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])

torch.set_default_dtype(torch.float64)

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

data_items = data_ent.items()
np.random.shuffle(list(data_items))
data_ent = OrderedDict(data_items)

data_items = data_prop.items()
np.random.shuffle(list(data_items))
data_prop = OrderedDict(data_items)

print ("Number of entity pairs:", len(data_ent))
print ("Number of property pairs:", len(data_prop))

torch.set_default_dtype(torch.float64)

index=0
ontologies_in_alignment = [tuple([elem.split("/")[-1].split(".")[0] for elem in pair]) for pair in ontologies_in_alignment]
if ontology_split:
    # We split on the ontology-pair level
    step = int(len(ontologies_in_alignment)/K)
    
    val_onto = ontologies_in_alignment[len(ontologies_in_alignment)-step+1:]
    train_data_ent = {elem: data_ent[elem] for elem in data_ent if tuple([el.split("#")[0] for el in elem]) not in val_onto}
    val_data_ent = {elem: data_ent[elem] for elem in data_ent if tuple([el.split("#")[0] for el in elem]) in val_onto}
    
    train_data_prop = {elem: data_prop[elem] for elem in data_prop if tuple([el.split("#")[0] for el in elem]) not in val_onto}
    val_data_prop = {elem: data_prop[elem] for elem in data_prop if tuple([el.split("#")[0] for el in elem]) in val_onto}
    
    train_data_t_ent = [key for key in train_data_ent if train_data_ent[key]]
    train_data_f_ent = [key for key in train_data_ent if not train_data_ent[key]]

    train_data_t_prop = [key for key in train_data_prop if train_data_prop[key]]
    train_data_f_prop = [key for key in train_data_prop if not train_data_prop[key]]

    val_data_t_ent = [key for key in val_data_ent if val_data_ent[key]]
    val_data_f_ent = [key for key in val_data_ent if not val_data_ent[key]]

    val_data_t_prop = [key for key in val_data_prop if val_data_prop[key]]
    val_data_f_prop = [key for key in val_data_prop if not val_data_prop[key]]

else:
    # We split on the mapping-pair level
    ratio = float(1/K)
    data_t_ent = {elem: data_ent[elem] for elem in data_ent if data_ent[elem]}
    data_f_ent = {elem: data_ent[elem] for elem in data_ent if not data_ent[elem]}

    data_t_prop = {elem: data_prop[elem] for elem in data_prop if data_prop[elem]}
    data_f_prop = {elem: data_prop[elem] for elem in data_prop if not data_prop[elem]}

    data_t_items_ent = list(data_t_ent.keys())
    data_f_items_ent = list(data_f_ent.keys())

    data_t_items_prop = list(data_t_prop.keys())
    data_f_items_prop = list(data_f_prop.keys())

    val_data_t_ent = data_t_items_ent[int((ratio*index)*len(data_t_ent)):int((ratio*index + ratio)*len(data_t_ent))]
    val_data_f_ent = data_f_items_ent[int((ratio*index)*len(data_f_ent)):int((ratio*index + ratio)*len(data_f_ent))]

    train_data_t_ent = data_t_items_ent[:int(ratio*index*len(data_t_ent))] + data_t_items_ent[int(ratio*(index+1)*len(data_t_ent)):]
    train_data_f_ent = data_f_items_ent[:int(ratio*index*len(data_f_ent))] + data_f_items_ent[int(ratio*(index+1)*len(data_f_ent)):]

    val_data_t_prop = data_t_items_prop[int((ratio*index)*len(data_t_prop)):int((ratio*index + ratio)*len(data_t_prop))]
    val_data_f_prop = data_f_items_prop[int((ratio*index)*len(data_f_prop)):int((ratio*index + ratio)*len(data_f_prop))]

    train_data_t_prop = data_t_items_prop[:int(ratio*index*len(data_t_prop))] + data_t_items_prop[int(ratio*(index+1)*len(data_t_prop)):]
    train_data_f_prop = data_f_items_prop[:int(ratio*index*len(data_f_prop))] + data_f_items_prop[int(ratio*(index+1)*len(data_f_prop)):]

np.random.shuffle(train_data_f_ent)
train_data_f_ent = train_data_f_ent[:max_false_examples]

np.random.shuffle(train_data_f_prop)
train_data_f_prop = train_data_f_prop[:max_false_examples]

# Oversampling to maintain 1:1 ratio between positives and negatives
train_data_t_ent = np.repeat(train_data_t_ent, ceil(len(train_data_f_ent)/len(train_data_t_ent)), axis=0)
train_data_t_ent = train_data_t_ent[:len(train_data_f_ent)].tolist()

if train_data_t_prop:
    train_data_t_prop = np.repeat(train_data_t_prop, ceil(len(train_data_f_prop)/len(train_data_t_prop)), axis=0)
    train_data_t_prop = train_data_t_prop[:len(train_data_f_prop)].tolist()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SiamHAN(emb_vals, max_types, max_paths, max_pathlen).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

print ("Starting training...")

for epoch in range(num_epochs):
    inputs_pos_ent, nodes_pos_ent, targets_pos_ent = generate_input(train_data_t_ent, 1, neighbours_dicts_ent)
    inputs_neg_ent, nodes_neg_ent, targets_neg_ent = generate_input(train_data_f_ent, 0, neighbours_dicts_ent)
    inputs_pos_prop, nodes_pos_prop, targets_pos_prop = generate_input(train_data_t_prop, 1, neighbours_dicts_prop)
    inputs_neg_prop, nodes_neg_prop, targets_neg_prop = generate_input(train_data_f_prop, 0, neighbours_dicts_prop)

    inputs_all_ent = list(inputs_pos_ent) + list(inputs_neg_ent)
    targets_all_ent = list(targets_pos_ent) + list(targets_neg_ent)
    nodes_all_ent = list(nodes_pos_ent) + list(nodes_neg_ent)
    
    all_inp_ent = list(zip(inputs_all_ent, targets_all_ent, nodes_all_ent))
    all_inp_shuffled_ent = random.sample(all_inp_ent, len(all_inp_ent))
    inputs_all_ent, targets_all_ent, nodes_all_ent = list(zip(*all_inp_shuffled_ent))

    inputs_all_prop = list(inputs_pos_prop) + list(inputs_neg_prop)
    targets_all_prop = list(targets_pos_prop) + list(targets_neg_prop)
    nodes_all_prop = list(nodes_pos_prop) + list(nodes_neg_prop)

    if len(inputs_all_prop) == 0:
        max_prop_len = 0
    else:
        max_prop_len = np.max([[[len(elem) for elem in prop] for prop in elem_pair] 
        for elem_pair in inputs_all_prop])
    
    all_inp_prop = list(zip(inputs_all_prop, targets_all_prop, nodes_all_prop))
    all_inp_shuffled_prop = random.sample(all_inp_prop, len(all_inp_prop))
    if all_inp_shuffled_prop:
        inputs_all_prop, targets_all_prop, nodes_all_prop = list(zip(*all_inp_shuffled_prop))
    else:
        inputs_all_prop, targets_all_prop, nodes_all_prop = [], [], []

    batch_size = min(batch_size, len(inputs_all_ent))
    num_batches = int(ceil(len(inputs_all_ent)/batch_size))
    batch_size_prop = int(ceil(len(inputs_all_prop)/num_batches))

    for batch_idx in range(num_batches):
        batch_start_ent = batch_idx * batch_size
        batch_end_ent = (batch_idx+1) * batch_size
        batch_start_prop = batch_idx * batch_size_prop
        batch_end_prop = (batch_idx+1) * batch_size_prop
        
        inputs_ent = np.array(to_feature(inputs_all_ent[batch_start_ent: batch_end_ent]))
        targets_ent = np.array(targets_all_ent[batch_start_ent: batch_end_ent])
        nodes_ent = np.array(nodes_all_ent[batch_start_ent: batch_end_ent])

        inputs_prop = np.array(pad_prop(inputs_all_prop[batch_start_prop: batch_end_prop], max_prop_len))
        targets_prop = np.array(targets_all_prop[batch_start_prop: batch_end_prop])
        nodes_prop = np.array(nodes_all_prop[batch_start_prop: batch_end_prop])
        
        targets = np.concatenate((targets_ent, targets_prop), axis=0)
        
        inp_elems = torch.LongTensor(inputs_ent).to(device)
        node_elems = torch.LongTensor(nodes_ent).to(device)
        targ_elems = torch.DoubleTensor(targets).to(device)

        inp_props = torch.LongTensor(inputs_prop).to(device)
        node_props = torch.LongTensor(nodes_prop).to(device)

        optimizer.zero_grad()
        outputs = model(node_elems, inp_elems, node_props, inp_props, max_prop_len)

        loss = F.mse_loss(outputs, targ_elems)
        loss.backward()
        optimizer.step()

        if batch_idx%5000 == 0:
            print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))  

print ("Training complete!")

model.eval()

print ("Optimizing threshold...")
optimize_threshold()

threshold_results_mean = {el: np.mean(threshold_results[el], axis=0) for el in threshold_results}    
threshold = max(threshold_results_mean.keys(), key=(lambda key: threshold_results_mean[key][2]))
print(threshold)
model.threshold=nn.Parameter(torch.DoubleTensor([threshold]))

torch.save(model.state_dict(), prefix_path+"model_2.pt")

print ("Done. Saved model at {}".format(model_path))


Prefix path:  drive/MyDrive/Thesis_OM/
Ontologies being aligned are:  [('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/cmt.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/confOf.owl'), ('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/conference.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/iasted.owl'), ('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/edas.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/sigkdd.owl'), ('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/edas.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/iasted.owl'), ('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/edas.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/ekaw.owl'), ('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/confOf.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/edas.owl'), ('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/conference.owl', 'drive/MyDrive/Thesis_OM

In [15]:

threshold = model.threshold.data.cpu().numpy()[0]

In [ ]:
threshold

In [ ]:
model.max_types.data.cpu().numpy()[0]

In [ ]:
pretrained_dict = torch.load("drive/MyDrive/Thesis_OM/model.pt", map_location=torch.device(device))

In [ ]:
pretrained_dict

In [16]:
import itertools
from xml.dom import minidom
from urllib.request import urlopen

flatten = lambda l: [item for sublist in l for item in sublist]

# Define class to parse ontology
class Ontology():
    def __init__(self, ontology):
        '''
        Instantiates an Ontology object. 
        Args:
            ontology: Path to ontology file
        Returns:
            Parsed Ontology object
        '''
        self.ontology = ontology
        if ontology.startswith("https://") or ontology.startswith("http://"):
            self.ontology_obj = minidom.parse(urlopen(ontology + "/"))
        else:
            self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement # root node
        
        self.languages = []
        self.construct_mapping_dict()
        # Detects language of ontology by taking max of all lang attributes in labels
        self.detect_language()
        
        # Dict that records the immediate "subclass_of" parent of any entity
        self.parents_dict = {}

        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    

    ### TODO missing codes ###
    def construct_mapping_dict(self):
        '''
        Constructs ID to label mapping dict for ontologies where 
        entities are identified by IDs.
        '''
        elements = self.root.getElementsByTagName("owl:Class") + self.root.getElementsByTagName("owl:ObjectProperty") + self.root.getElementsByTagName("owl:DatatypeProperty")
        self.mapping_dict = {self.extract_ID(el, False): self.return_label(el) for el in elements if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
    
    def return_label(self, el):
        '''
        Returns label of an element, and also detects language of the label
        '''
        label_element = self.get_child_node(el, "rdfs:label")[0]
        lang = label_element.getAttribute("xml:lang")
        if lang:
            self.languages.append(lang)
        return label_element.firstChild.nodeValue

    def detect_language(self):
        if self.languages:
            self.language = max(set(self.languages), key=self.languages.count)
        else:
            self.language = "en"

    def get_child_node(self, element, tag):
        '''
        Returns child node with a specific tag name given DOM element 
        Args:
            element: DOM parent element
            tag: Name of tag of child
        Returns:
            DOM child element 
        '''
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        '''
        Checks whether DOM element has attribute with a particular value
        Args:
            element: DOM parent element
            attribute: Attribute ID
            value: Required value of attribute
        Returns:
            boolean
        '''
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self, check_coded=False):
        '''
        Returns subclass triples
        '''
        subclasses = self.get_subclasses(check_coded)
        return [(b,a,c,d) for (a,b,c,d) in subclasses]
    
    def parse_triples(self, union_flag=0, subclass_of=True, check_coded=False):
        '''
        Parses ontology to obtain object property, data property and subclass triples 
        Args:
            union_flag: 0, if classes containing union of n classes are to be denoted 
            as a single class or n separate classes, else 1
            subclass_of: Determines if subclass triples should be returned or not
            check_coded: Determines if element should be queried for its label while extracting ID
        Returns:
            list of 4-tuples of the form (a,b,c,d) where d is the type of property c is.
        '''
        obj_props = [(prop, "Object Property") for prop in self.object_properties]
        data_props = [(prop, "Datatype Property") for prop in self.data_properties]
        props = obj_props + data_props
        all_triples = []
        for prop, prop_type in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                # Either domain is not present or range is not present
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop), prop_type) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop), prop_type))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples(check_coded))
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True, check_coded=False):
        '''
        Wrapper on top of parse_triples
        '''
        return self.parse_triples(union_flag, subclass_of, check_coded)

    def parse_subclasses(self):
        '''
        Parses ontology to obtain subclass triples
        Returns:
            list of subclass triples of the form (a,b,c,d)
        '''
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                # Subclass of with inline IDs
                subclass_pairs.append((el, el.parentNode, "subclass_of", "Subclass"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    # Subclass of relations with owl Restrictions
                    restriction = el.getElementsByTagName("owl:Restriction")
                    if not restriction:
                        continue
                    prop = self.get_child_node(restriction[0], "owl:onProperty")
                    some_vals = self.get_child_node(restriction[0], "owl:someValuesFrom")
                    
                    if not prop or not some_vals:
                        continue
                    try:
                        if self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                        else:
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                    except Exception as e:
                        try:
                            if not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                            elif not self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                class_vals = self.get_child_node(some_vals[0], "owl:Class")
                                subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                        except Exception as e:
                            continue
                else:
                    if self.extract_ID(level1_class[0]):
                        # Subclass label under a level 1 tag
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of", "Subclass"))
                    else:
                        continue
        return subclass_pairs
        
    def get_subclasses(self, check_coded=False):
        '''
        Extracts entity ID from parsed subclass triples
        '''
        subclasses = [(self.extract_ID(a, not check_coded), self.extract_ID(b, not check_coded), c, d) for (a,b,c,d) in self.subclasses]
        self.parents_dict = {}
        for (a,b,c,d) in subclasses:
            if c == "subclass_of" and a!="Thing" and b!="Thing":
                if b not in self.parents_dict:
                    self.parents_dict[b] = [a]
                else:
                    self.parents_dict[b].append(a)
        return [el for el in subclasses if el[0] and el[1] and el[2] and el[0]!="Thing" and el[1]!="Thing"]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ns(self):
        '''
        Extracts namespace of an ontology
        '''
        ns = self.ontology_obj.getElementsByTagName("rdf:RDF")[0].getAttribute("xmlns")
        if ns[-1] == "#":
            return ns
        return self.ontology_obj.doctype.entities.item(0).firstChild.nodeValue

    def extract_ID(self, element, check_coded = True):
        '''
        Returns ID for a parsed DOM element. In ontologies where classes are represented by 
        numerical IDs, it returns the label (stored in mapping_dict)
        '''
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1].split(";")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")

    def parse_classes(self):
        '''
        Parse all entities, including domain and range entities in ontology
        '''
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-2] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        '''
        Parse only classes
        '''
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        '''
        Parse all datatype properties, including functional and inverse functional datatype properties
        '''
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        '''
        Parse all object properties, including functional and inverse functional object properties
        '''
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        '''
        Wrapper to obtain object properties and parse them to return property IDs 
        '''
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        '''
        Wrapper to obtain data properties and parse them to return property IDs 
        '''
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))


## Test

In [17]:
onto_name_list = []
for i, file in enumerate(os.listdir("drive/MyDrive/Thesis_OM/datasets/conference/alignments")):
  onto_name_list.append(file.split(".")[0].split("-"))


In [21]:
import configparser, sys, logging, random, os, pickle
import numpy as np
from collections import OrderedDict
from math import ceil
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F


for model_name in ["model_2.pt"]:

  model_path ="drive/MyDrive/Thesis_OM/"+model_name
  for ont_name1, ont_name2 in onto_name_list:
    prefix_path_onto = "drive/MyDrive/Thesis_OM/datasets/conference/ontologies/"
    ont_name1, ont_name2  = prefix_path_onto+ont_name1+".owl", prefix_path_onto+ont_name2+".owl"
    if ont_name1.endswith("/"):
        ont_name1 = ont_name1[:-1]
    if ont_name2.endswith("/"):
        ont_name2 = ont_name2[:-1]

    prefix_path = "drive/MyDrive/Thesis_OM/"#"/".join(os.path.dirname(os.path.abspath(__file__)).split("/")[:-1]) + "/"
    outfolder = prefix_path+model_name.split(".")[0]+"/"



    # Read `config.ini` and initialize parameter values
    config = configparser.ConfigParser()
    config.read(prefix_path + 'config.ini')

    logging.info("Prefix path: ", prefix_path)

    # Initialize variables from config
    quick_mode = str(config["General"]["quick_mode"])

    #model_path = prefix_path + str(config["Paths"]["load_model_path"])
    #output_path = prefix_path + str(config["Paths"]["output_folder"])
    cached_embeddings_path = prefix_path + str(config["Paths"]["embedding_cache_path"])
    spellcheck = config["Preprocessing"]["has_spellcheck"] == "True"

    max_paths = int(config["Parameters"]["max_paths"])
    max_pathlen = int(config["Parameters"]["max_pathlen"])
    bag_of_neighbours = config["Parameters"]["bag_of_neighbours"] == "True"
    weighted_sum = config["Parameters"]["weighted_sum"] == "True"
    # print(model_settings.loc[model_settings["model_name"]==model_name]["batch_size"])
    batch_size =16# model_settings.loc[model_settings["model_name"]==model_name]["batch_size"].values[0] #int(config["Hyperparameters"]["batch_size"])
   
    batch_size = int(batch_size)
    test_ontologies = [tuple([ont_name1, ont_name2])]
    print(test_ontologies)
    # Preprocessing and parsing input data for testing

    preprocessing = GraphParser(test_ontologies)
    test_data_ent, test_data_prop, emb_indexer_new, emb_indexer_inv_new, emb_vals_new, neighbours_dicts_ent, neighbours_dicts_prop, max_types = preprocessing.process(spellcheck)

    if os.path.isfile(cached_embeddings_path):
        logging.info("Found cached embeddings...")
        emb_indexer_cached, emb_indexer_inv_cached, emb_vals_cached = pickle.load(open(cached_embeddings_path, "rb"))
    else:
        emb_indexer_cached, emb_indexer_inv_cached, emb_vals_cached = {}, {}, []

    emb_vals, emb_indexer, emb_indexer_inv = list(emb_vals_cached), dict(emb_indexer_cached), dict(emb_indexer_inv_cached)

    s = set(emb_indexer.keys())
    idx = len(emb_indexer_inv)
    for term in emb_indexer_new:
        if term not in s:
            emb_indexer[term] = idx
            emb_indexer_inv[idx] = term
            emb_vals.append(emb_vals_new[emb_indexer_new[term]])
            idx += 1

    def count_non_unk(elem):
        return len([l for l in elem if l!="<UNK>"])

    def generate_data_neighbourless(elem_tuple):
        return [emb_indexer[elem] for elem in elem_tuple]

    def embedify(seq):
        for item in seq:
            if isinstance(item, list):
                yield list(embedify(item))
            else:
                yield emb_indexer[item]

    def generate_data(elem_tuple, neighbours_dicts):
        return list(embedify([neighbours_dicts[elem] for elem in elem_tuple]))

    def to_feature(inputs):
        inputs_lenpadded = [[[[path[:max_pathlen] + [0 for i in range(max_pathlen -len(path[:max_pathlen]))]
                                        for path in nbr_type[:max_paths]]
                                    for nbr_type in ent[:max_types]]
                                for ent in elem]
                            for elem in inputs]
        inputs_pathpadded = [[[nbr_type + [[0 for j in range(max_pathlen)]
                                for i in range(max_paths - len(nbr_type))]
                                for nbr_type in ent] for ent in elem]
                            for elem in inputs_lenpadded]
        return inputs_pathpadded

    def pad_prop(inputs):
        inputs_padded = [[[elem + [0 for i in range(max_prop_len - len(elem))]
                            for elem in prop]
                        for prop in elem_pair]
                    for elem_pair in inputs]
        return inputs_padded

    def generate_input(elems, neighbours_dicts):
        inputs, nodes = [], []
        
        global direct_inputs
        for elem in list(elems):
            try:
                inputs.append(generate_data(elem, neighbours_dicts))
                nodes.append(generate_data_neighbourless(elem))
            except Exception as e:
                direct_inputs.append(generate_data_neighbourless(elem))
        return inputs, nodes

    def write_results():
        ont_name_parsed1 = Ontology(ont_name1).extract_ns()
        ont_name_parsed2 = Ontology(ont_name2).extract_ns()
        ont_name1_pre = ont_name1 if (ont_name1.startswith("http://") or ont_name1.startswith("https://")) else "file://" + ont_name1
        ont_name2_pre = ont_name2 if (ont_name2.startswith("http://") or ont_name2.startswith("https://")) else "file://" + ont_name2
        rdf = \
        """<?xml version='1.0' encoding='utf-8' standalone='no'?>
    <rdf:RDF xmlns='http://knowledgeweb.semanticweb.org/heterogeneity/alignment#'
            xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'
            xmlns:xsd='http://www.w3.org/2001/XMLSchema#'
            xmlns:align='http://knowledgeweb.semanticweb.org/heterogeneity/alignment#'>
    <Alignment>
      <xml>yes</xml>
      <level>0</level>
      <type>**</type>
      <onto1>
        <Ontology rdf:about="{}">
          <location>{}</location>
        </Ontology>
      </onto1>
      <onto2>
        <Ontology rdf:about="{}">
          <location>{}</location>
        </Ontology>
      </onto2>""".format(ont_name_parsed1.split("#")[0], ont_name1_pre, ont_name_parsed2.split("#")[0], ont_name2_pre)
        for (a,b,score) in final_list:
            mapping = """
      <map>
        <Cell>
          <entity1 rdf:resource='{}'/>
          <entity2 rdf:resource='{}'/>
          <relation>=</relation>
          <measure rdf:datatype='http://www.w3.org/2001/XMLSchema#float'>{}</measure>
        </Cell>
      </map>""".format(ont_name_parsed1 + "#".join(a.split("#")[1:]), ont_name_parsed2 + "#".join(b.split("#")[1:]), score)
            rdf += mapping
        rdf += """
    </Alignment>
    </rdf:RDF>"""
        return rdf

    torch.set_default_dtype(torch.float64)

    torch.manual_seed(0)
    np.random.seed(0)
    random.seed(0)

    np.random.shuffle(test_data_ent)
    np.random.shuffle(test_data_prop)

    torch.set_default_dtype(torch.float64)

    logging.info ("Loading trained model....")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    pretrained_dict = torch.load(model_path, map_location=torch.device(device))
    
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k!="name_embedding.weight"}
    
    #max_types = len([key for key in pretrained_dict.keys() if key.startswith("w_")]) + 1
    max_types = int(model.max_types.data.cpu().numpy()[0])
    model = SiamHAN(emb_vals, max_types, max_paths, max_pathlen).to(device)
    model_dict = model.state_dict()

    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    threshold = model.threshold.data.cpu().numpy()[0]
    print(threshold)
    logging.info ("Model loaded successfully!")

    logging.info ("Optimum Threshold: {}".format(threshold))

    model.eval()
    logging.info ("Length of test data(ent): {} test data(prop):{}".format(len(test_data_ent), len(test_data_prop)))

    all_results = OrderedDict()    
    direct_inputs = []
    with torch.no_grad():
        inputs_all_ent, nodes_all_ent = generate_input(test_data_ent, neighbours_dicts_ent)
        inputs_all_prop, nodes_all_prop = generate_input(test_data_prop, neighbours_dicts_prop)
        all_inp = list(zip(inputs_all_ent, nodes_all_ent))
        all_inp_shuffled = random.sample(all_inp, len(all_inp))

        inputs_all_ent, nodes_all_ent = list(zip(*all_inp_shuffled))

        all_inp = list(zip(inputs_all_prop, nodes_all_prop))
        all_inp_shuffled = random.sample(all_inp, len(all_inp))
        inputs_all_prop, nodes_all_prop = list(zip(*all_inp_shuffled))
        
        max_prop_len = np.max([[[len(elem) for elem in prop] for prop in elem_pair]
            for elem_pair in inputs_all_prop])
        logging.info ("Max prop len: ", max_prop_len)
        batch_size = min(batch_size, len(inputs_all_ent))
        num_batches = int(ceil(len(inputs_all_ent)/batch_size))
        batch_size_prop = int(ceil(len(inputs_all_prop)/num_batches))

        logging.info ("Num batches: {} Batch size (prop): {}".format(num_batches, batch_size_prop))
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size
            batch_start_prop = batch_idx * batch_size_prop
            batch_end_prop = (batch_idx+1) * batch_size_prop
            inputs_ent = np.array(to_feature(inputs_all_ent[batch_start: batch_end]))
            nodes_ent = np.array(nodes_all_ent[batch_start: batch_end])

            inputs_prop = np.array(pad_prop(inputs_all_prop[batch_start_prop: batch_end_prop]))
            nodes_prop = np.array(nodes_all_prop[batch_start_prop: batch_end_prop])

            inp_ents = torch.LongTensor(inputs_ent).to(device)
            node_ents = torch.LongTensor(nodes_ent).to(device)
            inp_props = torch.LongTensor(inputs_prop).to(device)
            node_props = torch.LongTensor(nodes_prop).to(device)
            outputs = model(node_ents, inp_ents, node_props, inp_props, max_prop_len)
            outputs = [el.item() for el in outputs]

            for idx, pred_elem in enumerate(outputs):
                if idx < len(nodes_ent):
                    ent1 = emb_indexer_inv[nodes_ent[idx][0]]
                    ent2 = emb_indexer_inv[nodes_ent[idx][1]]
                else:
                    ent1 = emb_indexer_inv[nodes_prop[idx-len(nodes_ent)][0]]
                    ent2 = emb_indexer_inv[nodes_prop[idx-len(nodes_ent)][1]]
                if (ent1, ent2) in all_results:
                    logging.info ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (round(pred_elem, 3), pred_elem>=threshold)
        
        logging.info ("Len (direct inputs): ", len(direct_inputs))
        for idx, direct_input in enumerate(direct_inputs):
            ent1 = emb_indexer_inv[direct_input[0]]
            ent2 = emb_indexer_inv[direct_input[1]]
            sim = cos_sim(emb_vals[direct_input[0]], emb_vals[direct_input[1]])
            all_results[(ent1, ent2)] = (round(sim, 3), pred_elem>=threshold)
        
    final_list = [(elem[0], elem[1], str(all_results[elem][0])) for elem in all_results if all_results[elem][1]]

    ont_name_parsed1 = Ontology(ont_name1).extract_ns().split("/")[-1].split("#")[0].rsplit(".", 1)[0]
    ont_name_parsed2 = Ontology(ont_name2).extract_ns().split("/")[-1].split("#")[0].rsplit(".", 1)[0]

    f = "HAN1-"+ont_name_parsed1.lower() + "-" + ont_name_parsed2.lower() + ".rdf"

    open(outfolder + f, "w+").write(write_results())

    logging.info ("The final alignment file can be found below: ")
    print("file://" + outfolder + f)


[('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/cmt.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/confOf.owl')]
0.898
file://drive/MyDrive/Thesis_OM/model_2/HAN1-cmt-confof.rdf
[('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/conference.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/iasted.owl')]
0.898
file://drive/MyDrive/Thesis_OM/model_2/HAN1-conference-iasted.rdf
[('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/edas.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/sigkdd.owl')]
0.898
file://drive/MyDrive/Thesis_OM/model_2/HAN1-edas-sigkdd.rdf
[('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/edas.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/iasted.owl')]
0.898
file://drive/MyDrive/Thesis_OM/model_2/HAN1-edas-iasted.rdf
[('drive/MyDrive/Thesis_OM/datasets/conference/ontologies/edas.owl', 'drive/MyDrive/Thesis_OM/datasets/conference/ontologies/ekaw.owl')]
0.898
file://drive/MyDrive/Thesis